<a href="https://colab.research.google.com/github/MahdiTheGreat/Intro-to-language-modeling/blob/main/Retrieval_augmented_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_huggingface
!pip install -qU langchain-text-splitters
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.7 MB/s eta 0:00:00


In [2]:
langchain_model_name = "sentence-transformers/all-mpnet-base-v2"

In [3]:
!wget https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json

--2024-12-10 10:12:48--  https://raw.githubusercontent.com/pubmedqa/pubmedqa/refs/heads/master/data/ori_pqal.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2584787 (2.5M) [text/plain]
Saving to: ‘ori_pqal.json’

ori_pqal.json       100%[===================>]   2.46M  --.-KB/s    in 0.03s   

2024-12-10 10:12:48 (93.2 MB/s) - ‘ori_pqal.json’ saved [2584787/2584787]



We collect two datasets:

    ‘questions’: the questions with corresponding gold long answer, gold document ID, and year.
    ‘documents’: the abstracts (contexts+long_answer concatenated), and year.


In [4]:
import pandas as pd
tmp_data = pd.read_json("/content/ori_pqal.json").T
# some labels have been defined as "maybe", only keep the yes/no answers
tmp_data = tmp_data[tmp_data.final_decision.isin(["yes", "no"])]

documents = pd.DataFrame({"abstract": tmp_data.apply(lambda row: (" ").join(row.CONTEXTS+[row.LONG_ANSWER]), axis=1),
             "year": tmp_data.YEAR})
questions = pd.DataFrame({"question": tmp_data.QUESTION,
             "year": tmp_data.YEAR,
             "gold_label": tmp_data.final_decision,
             "gold_context": tmp_data.LONG_ANSWER,
             "gold_document_id": documents.index})

For an example of a query:

In [5]:
questions.iloc[0].question

'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?'

For an example of a document to leverage for the queries:

In [6]:
documents.iloc[0].abstract

'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTracker Red CMXRos a

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=langchain_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

In [9]:
texts = text_splitter.create_documents([documents.iloc[0].abstract])
print(texts[0])
print(texts[1])
text_splitter.split_text(documents.iloc[0].abstract)[:2]

page_content='Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant'
page_content='The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The'


['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant',
 'The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The']

If you want to get best in-class automated tracing of your model calls you can also set your LangSmith API key by uncommenting below:

In [10]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

In [11]:
documents.iloc[0:10].abstract

,abstract
21645374,Programmed cell death (PCD) is the regulated d...
16418930,Assessment of visual acuity depends on the opt...
9488747,Apparent life-threatening events in infants ar...
17208539,The transanal endorectal pull-through (TERPT) ...
10808977,Telephone counseling and tailored print commun...
23831910,"From March 2007 to January 2011, 88 DBE proced..."
26852225,Anchoring vignettes are brief texts describing...
17113061,Although observational data support an inverse...
10966337,We evaluated the usefulness of a short stay or...
25432938,The objective of the current study is to deter...


In [12]:
from uuid import uuid4

from langchain_core.documents import Document

docs=[]

for doc in documents.iloc[0:10].abstract:
    temp_doc = Document(
        page_content=doc,
        metadata={"source": "tweet"},
        id=uuid4(),
    )
    docs.append(temp_doc)

In [13]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_documents(docs, hf)
#retriever = vector_store.as_retriever(...)

In [14]:
results = vector_store.similarity_search_with_score(
    "What is programmed cell death?", k=3
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=1.262025] Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoTrac

In [19]:
from huggingface_hub import login
login()

In [23]:
! huggingface-cli download meta-llama/Llama-3.2-1B-Instruct --local-dir Llama-3.2-3B-Instruct

Fetching 13 files: 100% 13/13 [00:47<00:00,  3.63s/it]
/content/Llama-3.2-3B-Instruct


In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("Llama-3.2-3B-Instruct")
model_hf = AutoModelForCausalLM.from_pretrained("Llama-3.2-3B-Instruct")

In [36]:
retriever = vector_store.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer,
    max_new_tokens=512,
    device="cuda"
    )

# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

response = retrieval_chain.invoke("Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?")

# Assuming the output is a dictionary or a similar structure:
if isinstance(response, dict):
    for key, value in response.items():
        print(f"{key}: {value}")
elif isinstance(response, list):
    for idx, item in enumerate(response):
        print(f"Item {idx + 1}: {item}")
else:
    print(response)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Human: Answer the question based only on the following context:
[Document(metadata={'source': 'tweet'}, page_content='Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cel

In [62]:
from tqdm import tqdm
template = """Answer the question, only with yes or no, based only on the following context:
{context}
Do not write anything else. Do not explain your answer.
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# Create a pipeline for text generation
pipe = pipeline(
    "text-generation",
    model=model_hf,
    tokenizer=tokenizer,
    max_new_tokens=10,
    device="cuda"
    )

# Wrap the pipeline in a LangChain HuggingFacePipeline object
llm = HuggingFacePipeline(pipeline=pipe)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Initialize a counter for correct answers
correct_count = 0
q_a = zip(questions.question, questions.gold_label)
print(len(questions.question))
for question, gold_label in q_a:
    answer = retrieval_chain.invoke(question)
    print(type(answer))
    y_n = answer.splitlines()[-1].split()[0]
    # Check if the answer is "yes" or "no" and matches the gold label
    if gold_label.lower() in answer[-1].lower():
        print("Correct")
        correct_count += 1
    else:
        print("Incorrect")

# Print the total number of correct answers
print(f"Total correct answers: {correct_count}")




Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


890
<class 'str'>
Incorrect


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


<class 'str'>
Incorrect


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


KeyboardInterrupt: 